<a href="https://colab.research.google.com/github/SofiaRguez1020/Topicos_Industria_I/blob/main/Proyecto_SofiaRodriguez__AlejandraTrujillo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final: *MiBici*

**Nombre:** Sofia Daniela Rodriguez Saenz

**e-mail:** sofia.rodriguez5540@alumnos.udg.mx

**Nombre:** Alejandra Elizabeth Trujillo Navarro

**e-mail:** alejandra.trujillo2826@alumnos.udg.mx


# Descripción del proyecto
En este proyecto nos centraremos en el análisis y la visualización de datos
provenientes del sistema de bicicletas compartidas, el cual es una programa
ecológica y sostenible. A través de la recolección y el estudio de datos detallados sobre el uso de las bicicletas, estaciones, y otros factores relevantes como edad y género, este proyecto busca ofrecer detalles relevantes para mejorar el servicio de MiBici.
Al analizar cómo, cuándo y dónde se utilizan las bicicletas compartidas, podemos obtener detalles sobre las necesidades y preferencias de los usuarios, lo que a su vez puede guiar mejoras en la infraestructura, la distribución de las estaciones y la promoción de hábitos de transporte más sostenibles.
## Capa 1: Panel de control para generación del dataset
El propósito de esta capa es permitir al usuario generar o filtrar el conjunto de datos que será analizado. Se utilizarán los datos abiertos de MiBici, disponibles en la siguiente liga: https://www.mibici.net/es/datos-abiertos/. Será posible seleccionar rangos de fechas, estaciones específicas de bicicletas tanto de retiro como de
arribo, y segmentar los datos por género y edad de los usuarios.
## Capa 2: Representación gráfica del dataset
En esta capa, se presentará una representación gráfica del conjunto de datos
filtrado o generado en la Capa 1. Exhibiremos un mapa detallado de la ciudad,
señalando la ubicación de las estaciones y su respectivo número identificativo.
Además, incorporaremos una codificación por colores para destacar las estaciones de mayor uso.
## Capa 3: Panel de Control para Selección de Métricas y
Visualización de Resultados
Esta capa permitiría al usuario seleccionar diferentes métricas para analizar el dataset y visualizar los resultados de manera gráfica.
* Distancia total recorrida: Un histograma o gráfico de líneas mostrando la
distancia total recorrida por mes.
* Duración promedio de los viajes: Gráficos de barras que muestren la duración promedio de los viajes por estación.
* Densidad de uso por hora del día: Gráficos de líneas que muestren cómo
varía el uso de las bicicletas a lo largo del día, identificando las horas pico.

# MODULES

In [50]:
import errno
import requests
import os
import pandas as pd
import panel as pn
import panel.widgets as pnw
from datetime import datetime

pn.extension('plotly')

# CLASSES

# FUNCTIONS

In [19]:
def getDF(path):
  df = pd.DataFrame()
  #Iterar sobre los archivos contenidos en el folder
  for archivo in os.listdir(path):
    #Extraemos el nombre del mes
    mes = archivo.split('.')[0]

    #Guardamos datos del archivo en un df temporal
    temp_df = pd.read_csv(os.path.join(path, archivo), encoding='iso-8859-1')
    temp_df['Mes'] = mes
    #combinamos los dataframes
    df = pd.concat([df, temp_df], ignore_index=True)
  return df



#Obtener datos de MiBici

In [8]:
#Creamos una carpeta para guardar los archivos
try:
    os.mkdir('MiBici')
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
# URL donde se encuentran guardados los archivos de Ecobici
urls = {'Enero':r'https://www.mibici.net/site/assets/files/43035/datos_abiertos_2023_03.csv',
        'Febrero': r'https://www.mibici.net/site/assets/files/40197/datos_abiertos_2023_02.csv',
        'Marzo': r'https://www.mibici.net/site/assets/files/43035/datos_abiertos_2023_03.csv',
        'Abril': r'https://www.mibici.net/site/assets/files/46758/datos_abiertos_2023_04.csv',
        'Mayo': r'https://www.mibici.net/site/assets/files/46759/datos_abiertos_2023_05.csv',
        'Junio': r'https://www.mibici.net/site/assets/files/48116/datos_abiertos_2023_06.csv',
        'Julio': r'https://www.mibici.net/site/assets/files/49448/datos_abiertos_2023_07.csv',
        'Agosto': r'https://www.mibici.net/site/assets/files/51295/datos_abiertos_2023_08.csv',
        'Septiembre': r'https://www.mibici.net/site/assets/files/53344/datos_abiertos_2023_09.csv',
        'Octubre': r'https://www.mibici.net/site/assets/files/54935/datos_abiertos_2023_10.csv',
        'Noviembre': r'https://www.mibici.net/site/assets/files/57344/datos_abiertos_2023_11.csv',
        'Diciembre': r'https://www.mibici.net/site/assets/files/58715/datos_abiertos_2023_12.csv'
        }

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Realizar la solicitud GET para obtener archivo
#Descargaremos cada uno de los archivos de 2023 ya que 2024 no esta completo aun
for mes, url in urls.items():
    url_tmp = url
    print("Descargando datos: ", url, " ....")
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        # Abrir el archivo en modo binario y escribir los datos descargados
        ruta = 'MiBici/'+mes+'.csv'
        with open(ruta, "wb") as archivo:
            archivo.write(response.content)
        print(f"Archivo guardado como {mes}.csv")
    else:
        print(f"Error al descargar el archivo. Código de respuesta: {response.status_code}")


Descargando datos:  https://www.mibici.net/site/assets/files/43035/datos_abiertos_2023_03.csv  ....
Archivo guardado como Enero.csv
Descargando datos:  https://www.mibici.net/site/assets/files/40197/datos_abiertos_2023_02.csv  ....
Archivo guardado como Febrero.csv
Descargando datos:  https://www.mibici.net/site/assets/files/43035/datos_abiertos_2023_03.csv  ....
Archivo guardado como Marzo.csv
Descargando datos:  https://www.mibici.net/site/assets/files/46758/datos_abiertos_2023_04.csv  ....
Archivo guardado como Abril.csv
Descargando datos:  https://www.mibici.net/site/assets/files/46759/datos_abiertos_2023_05.csv  ....
Archivo guardado como Mayo.csv
Descargando datos:  https://www.mibici.net/site/assets/files/48116/datos_abiertos_2023_06.csv  ....
Archivo guardado como Junio.csv
Descargando datos:  https://www.mibici.net/site/assets/files/49448/datos_abiertos_2023_07.csv  ....
Archivo guardado como Julio.csv
Descargando datos:  https://www.mibici.net/site/assets/files/51295/datos_ab

In [20]:
#Procederemos a guardar en 1 Dataframe y conmbinarlos en 1 solo archivo
data = getDF(r'MiBici/')

In [49]:
data['Origen_Id'].unique()

array([228, 200,  58,  38, 154, 324,  43,  52,  42,  12, 277,  35, 182,
        72, 263, 117, 272,  32,  50, 305,  26, 279, 281, 177,  86, 236,
       161,  10,  70, 323,  51, 306,  64,  49,  37,  78, 188,  16,  66,
        68,  54,  31, 158,  79, 257, 235, 203,  39, 247, 145,  36, 190,
       193,  94, 157, 218, 260,  61, 290,  14, 296,  73, 112,  75,  82,
        40, 108, 286, 304, 234,  76,  11,  80,  62, 227, 291, 326, 152,
       255, 180, 197, 142, 170,  28, 179, 275, 164,  84, 288, 169, 238,
       143,   9, 216, 266, 314,  30, 178, 256,  56,   6, 131,   5,  74,
       156, 198,  21, 321, 165, 150, 211, 231,   3,  33, 225,  93, 175,
        44, 243, 319,  53, 259, 311,  45,  88, 232, 271,  92, 268, 171,
       303, 276,  83, 280,  69, 201, 322, 315, 172, 199, 105, 265, 196,
       221, 251,   2, 327,  81, 307, 226, 262,  67, 162,  23,  91,  46,
        63, 174, 261,  24,  25, 137, 273, 285, 181, 284, 163,  87, 207,
       186,  19,  57, 269, 146, 173, 132, 246, 168,  17,  13, 20

#Creación del Dashboard

In [79]:
pn.extension()

#Definir layout de los widgets
estacion_id = pn.widgets.AutocompleteInput(name='ID estación', options=list(map(str, data['Origen_Id'].unique())), width = 320, align = 'center', margin=15)
genero = pn.widgets.ToggleGroup(name='Género', options=['M','F'], behavior="radio", width = 320, align='center')
anio_actual = datetime.now().year
edad = pn.widgets.IntSlider(name='Edad:', start=int(anio_actual-data['Año_de_nacimiento'].max()), end=int(anio_actual-data['Año_de_nacimiento'].min()), value=25, width = 320, align = 'center')
fecha_retiro = pn.widgets.DatetimePicker(name='Fecha y hora retiro', value=datetime(2023, 1, 1, 0, 0), width = 320, align = 'center')
fecha_arribo = pn.widgets.DatetimePicker(name='Fecha y hora arribo', value=datetime(2023, 1, 1, 0, 0), width = 320, align = 'center')


In [80]:
dashboard = pn.Row(
    pn.WidgetBox(pn.pane.Markdown("# Panel options", align='center'),
                 estacion_id,
                 genero,
                 edad,
                 fecha_retiro,
                 fecha_arribo,
                 width= 380,
                 height = 450, align = 'center'),
    pn.Column(width = 500,height = 500, align='center'),
    pn.Column(width = 500,height = 500, align='center'),
    margin=100
)


In [81]:
#Show dashboard embedded in colan
dashboard.servable()


Row(margin=100)
    [0] WidgetBox(align='center', height=450, sizing_mode='fixed', width=380)
        [0] Markdown(str, align='center')
        [1] AutocompleteInput(align='center', margin=15, name='ID estación', options=['228', '200', ...], width=320)
        [2] RadioButtonGroup(align='center', name='Género', options=['M', 'F'], value='M', width=320)
        [3] IntSlider(align='center', end=104, name='Edad:', start=1, value=25, width=320)
        [4] DatetimePicker(align='center', as_numpy_datetime64=False, name='Fecha y hora retiro', value=datetime.datetime(2023, ..., width=320)
        [5] DatetimePicker(align='center', as_numpy_datetime64=False, name='Fecha y hora arribo', value=datetime.datetime(2023, ..., width=320)
    [1] Column(align='center', height=500, sizing_mode='fixed', width=500)
    [2] Column(align='center', height=500, sizing_mode='fixed', width=500)

In [ ]:
#Show dashboard in localhost
dashboard.show()

Launching server at http://localhost:35439


## Notas

Link a codigo python unicamente se encuentra aqui:
https://github.com/SofiaRguez1020/Topicos_Industria_I/blob/main/ti_1_practica4_sofia_rodriguez.py